# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list.extend(glob.glob(os.path.join(root,'*')))
    


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


9372


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

7931


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifyks
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkifyks')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
# create user play of song log
# for key partition key use sessionId and itemInSession like clustering key
selectFromMusicLog = """select artist, song, length from music_log where sessionId = 338 and itemInSession = 4;"""
try:
    session.execute("""create table if not exists music_log (artist text, song text, length float, sessionId int, itemInSession int,
    primary key(sessionId, itemInSession))""")
except Exception as e:
    print(e)              

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_log(sessionId, itemInSession,artist, song, length)"
        query = query + " values (%s,%s,%s,%s,%s);"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
try:
    rows = session.execute(selectFromMusicLog)    
    for row in rows:
        print (row)
except Exception as e:
    print(e)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
# create song plays sessions to map user walk on playlists
# for partition key use UserId and sessionId, itemInSession for clustering keys
searchSession = """select artist, song, firstName, lastName from song_playlist_session where userid = 10 and sessionid = 182;"""
try:
    session.execute("""create table if not exists song_playlist_session (artist text, song text, sessionId int, itemInSession int, userId int, firstName text, lastName text,
    primary key(userid, sessionId, itemInSession))""")
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_playlist_session( userId, sessionId, itemInSession, song, artist, firstName, lastName)"
        query = query + " values (%s,%s,%s,%s,%s,%s,%s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),line[9], line[0],  line[1], line[4]))

In [14]:
# create user playlist to realize what s/he preferred
# song use for partition key and userid like clustering key 
searchSong = """select firstName, lastName from songs_playlist where song='All Hands Against His Own';"""
try:
    session.execute("""create table if not exists songs_playlist (song text, userId int, firstName text, lastName text,
    primary key(song, userid));""")
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into songs_playlist(song, userId, firstName, lastName)"
        query = query + " values (%s,%s,%s,%s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
try:
    rows = session.execute(searchSession)    
    for row in rows:
        print (row)
except Exception as e:
    print(e)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [17]:
try:
    rows = session.execute(searchSong)    
    for row in rows:
        print (row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
#Drop the table before closing out the sessions

In [19]:
try:
    rows = session.execute('drop table if exists music_library;')
except Exception as e:
    print(e)


try:
    rows = session.execute('drop table if exists song_playlist_session;')
except Exception as e:
    print(e)


try:
    rows = session.execute('drop table if exists songs_playlist;')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()